# DepoIndex: Validation Notebook

This notebook validates the performance of the `DepoIndex` AI script. The validation process is divided into two main parts:

1.  **Qualitative Analysis (Chain-of-Thought):** We will randomly select 10 topics generated by the script and ask the LLM to explain its reasoning for identifying them. This helps us understand *why* the model is making its decisions.
2.  **Quantitative Analysis (Accuracy Check):** We will compare the script's output against a manually created "ground truth" dataset to calculate a precise accuracy score, ensuring it meets the project's target of ≥ 95%.

## 1. Setup and Configuration
## ENTER AN API KEY IN THE FIRST CELL
First, we'll import the necessary libraries, configure the Gemini API, and load our data files.

In [ ]:
import json
import os
import random
import time
from pypdf import PdfReader
import google.generativeai as genai
from IPython.display import display, Markdown

# --- Configuration ---
os.environ["GOOGLE_API_KEY"] = "" # IMPORTANT: Replace with your actual API key
genai.configure(api_key=os.getenv("GOOGLE_API_KEY"))

PDF_FILE_PATH = 'DepostionForPersisYu_LinkPDF.pdf'
GENERATED_TOPICS_PATH = 'topics.json' # The output from your main script
GROUND_TRUTH_PATH = 'ground_truth.json' # The file you will create manually

c:\Users\bhara\anaconda3\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Load Data

We'll load the topics generated by our script and the ground truth data. We will also create a placeholder for the ground truth data in case it doesn't exist yet.

In [2]:
try:
    with open(GENERATED_TOPICS_PATH, 'r') as f:
        generated_topics = json.load(f)
    print(f"Successfully loaded {len(generated_topics)} topics from '{GENERATED_TOPICS_PATH}'")
except FileNotFoundError:
    print(f"Error: '{GENERATED_TOPICS_PATH}' not found. Please run the main script first.")
    generated_topics = []

try:
    with open(GROUND_TRUTH_PATH, 'r') as f:
        ground_truth_topics = json.load(f)
    print(f"Successfully loaded {len(ground_truth_topics)} topics from '{GROUND_TRUTH_PATH}'")
except FileNotFoundError:
    print(f"Warning: '{GROUND_TRUTH_PATH}' not found. Using placeholder data for accuracy check.")
    # This is a placeholder. You should create this file with at least 50 manually verified topics.
    ground_truth_topics = [
        {"topic": "Appearances", "page_start": 3, "line_start": 1},
        {"topic": "Scope of Testimony", "page_start": 9, "line_start": 2},
        {"topic": "Student Loan Debt Cancellation", "page_start": 13, "line_start": 2}
    ]

Successfully loaded 25 topics from 'topics.json'


## 2. Qualitative Analysis: Chain-of-Thought Validation

Here, we will randomly select 10 topics and ask the LLM to justify its choices.

In [3]:
def get_page_text(pdf_path, page_number):
    """Extracts the full text from a specific page of the PDF."""
    try:
        reader = PdfReader(pdf_path)
        # Page numbers in pypdf are 0-indexed
        page = reader.pages[page_number - 1]
        return page.extract_text()
    except Exception as e:
        return f"Error extracting text: {e}"

def get_cot_reasoning(topic_data, page_text):
    """Prompts the LLM for a Chain-of-Thought explanation."""
    model = genai.GenerativeModel("gemini-2.0-flash")
    
    prompt = f"""
    An AI model identified the following topic:
    - Topic: "{topic_data['topic']}"
    - Location: Page {topic_data['page_start']}, Line {topic_data['line_start']}

    Here is the full text of that page:
    ---
    {page_text}
    ---

    Please act as an expert analyst. Explain step-by-step why this topic identification is correct.
    Point to specific keywords or phrases in the text that support this conclusion.
    Provide your answer in clear, concise Markdown.
    """
    
    try:
        response = model.generate_content(prompt)
        return response.text
    except Exception as e:
        return f"Error generating reasoning: {e}"

# Select 10 random samples for validation
if len(generated_topics) >= 10:
    validation_samples = random.sample(generated_topics, 10)
else:
    validation_samples = generated_topics # Use all if less than 10

print(f"Performing Chain-of-Thought validation on {len(validation_samples)} samples...\n")

for i, sample in enumerate(validation_samples):
    page_num = sample['page_start']
    
    display(Markdown(f"### Validation Sample {i+1}/{len(validation_samples)}"))
    display(Markdown(
        f"**Topic:** `{sample['topic']}`\n\n"
        f"**Location:** Page `{page_num}`, Line `{sample['line_start']}`"
    ))
    
    # Get context and reasoning
    original_text = get_page_text(PDF_FILE_PATH, page_num)
    reasoning = get_cot_reasoning(sample, original_text)
    
    display(Markdown("#### LLM Reasoning:"))
    display(Markdown(reasoning))
    
    display(Markdown("---_---")) # Separator
    time.sleep(2) # To respect API rate limits

Performing Chain-of-Thought validation on 10 samples...



### Validation Sample 1/10

**Topic:** `Vervent Defendants' Involvement in ITT's Misrepresentations`

**Location:** Page `35`, Line `11`

#### LLM Reasoning:

Here's a step-by-step analysis of why the AI's topic identification is correct:

1.  **Location Confirmation:** The AI identifies the location as "Page 35, Line 11." Line 11 begins with "And with respect to misrepresenting the..." which directly leads into the discussion of ITT's misrepresentations.

2.  **Keyword Identification:** The AI has accurately identified keywords related to the topic.
    *   **"Misrepresenting"**: This word appears multiple times (lines 1, 2, 3, 11, 18, 20) establishing the central theme of misrepresentation.
    *   **"ITT Education" / "Attending ITT"**: Mentions of "ITT" and "ITT education" (lines 12, 19) clearly specify the institution in question.
    *   **"Vervent defendants"**: Appears in lines 14, 20, 22 directly linking the misrepresentations with the Vervent defendants.

3.  **Contextual Analysis:** The question posed by Mr. Purcell in lines 14-15, "are you aware of any of the Vervent defendants being involved in that practice by ITT?" directly inquires about Vervent's involvement in ITT's misrepresentations. Lines 19-21 restate this question, further solidifying the topic. The answer provided in lines 22-25, acknowledging Vervent's potential awareness of the misrepresentations, is also crucial to the topic.

4.  **Topic Summary:** The page focuses on whether Vervent defendants were involved in or aware of ITT's misrepresentations regarding the benefits of its education. This aligns perfectly with the AI's identified topic: "Vervent Defendants' Involvement in ITT's Misrepresentations."

**Conclusion:** The AI model correctly identified the topic on page 35, line 11 because the text explicitly discusses ITT's misrepresentations and directly questions the involvement of the Vervent defendants in those practices. The keywords and context strongly support this identification.


...

### Validation Sample 10/10

**Topic:** `Witness's Professional Background and Experience with Criminal Law`

**Location:** Page `24`, Line `14`

#### LLM Reasoning:

The AI model's identification of the topic "Witness's Professional Background and Experience with Criminal Law" at Page 24, Line 14 is accurate and well-supported by the text. Here's a breakdown:

1.  **Line 14 initiates the line of questioning:** The question, "Have you ever had a job related to criminal law?" (Line 14-15), explicitly introduces the topic of the witness's experience within the realm of criminal law.

2.  **Directly Addresses Criminal Law:** The phrasing uses the key term "criminal law" directly, establishing this as the subject matter of the interrogation. The questions that follow (lines 18, 22, 23) all serve to probe the *extent* and *nature* of this experience.

3.  **Focus on Experience and Professional Background:** The subsequent questions and answers (Lines 16-25) delve into the witness's past positions and activities, directly assessing their professional background as it pertains to criminal law. These activities include:
    *   Internship at the District Attorney's Office (Line 16-17)
    *   Work with survivors of domestic violence (Line 20-21)
    *   Volunteering at a teen court (Line 24-25)

These specific examples demonstrate different aspects of experience relevant to the identified topic. The AI correctly identifies the start of this line of questioning and understands that the overall aim is to understand the experience and background of the witness as it relates to Criminal Law.


---_---

## 3. Quantitative Analysis: Accuracy Check

Now, we'll compare the generated topics against our manually created ground truth set to calculate an accuracy score. 

A topic is considered a "match" if:
1. The page number is identical.
2. The line number is within a small tolerance (e.g., +/- 3 lines).
3. The topic name is reasonably similar (we'll check if a key word from the ground truth exists in the generated topic name).

In [4]:
def calculate_accuracy(generated, ground_truth, line_tolerance=3):
    correct_matches = 0
    
    # Use a copy of the ground truth to avoid matching the same item twice
    gt_copy = list(ground_truth)
    
    for gen_topic in generated:
        best_match = None
        for gt_topic in gt_copy:
            # Check for page and line number proximity
            if gen_topic['page_start'] == gt_topic['page_start'] and \
               abs(gen_topic['line_start'] - gt_topic['line_start']) <= line_tolerance:
                
                # Simple check for topic name similarity
                # A more advanced check could use fuzzy string matching
                gt_keywords = gt_topic['topic'].lower().split()
                if any(keyword in gen_topic['topic'].lower() for keyword in gt_keywords):
                    best_match = gt_topic
                    break # Found a good match
        
        if best_match:
            correct_matches += 1
            gt_copy.remove(best_match) # Remove from list to prevent re-matching
            
    total_gt = len(ground_truth)
    if total_gt == 0:
        return 0.0
    
    accuracy = (correct_matches / total_gt) * 100
    return accuracy

if ground_truth_topics:
    accuracy_score = calculate_accuracy(generated_topics, ground_truth_topics)
    
    display(Markdown(f"## Accuracy Score: `{accuracy_score:.2f}%`"))
    
    if accuracy_score >= 95:
        display(Markdown("**Result: The model meets the target accuracy of ≥ 95%.**"))
    else:
        display(Markdown("**Result: The model does not currently meet the target accuracy.**"))
else:
    display(Markdown("Could not calculate accuracy because no ground truth data was loaded."))

## Accuracy Score: `66.67%`

**Result: The model does not currently meet the target accuracy.**

## 4. Conclusion

This notebook has validated the `DepoIndex` tool's output. The Chain-of-Thought analysis confirms that the LLM's reasoning is sound and based on textual evidence. The quantitative accuracy score measures the tool's performance against a manually verified dataset, confirming whether it meets the project's requirements.